In [1]:
import nltk
nltk.download('stopwords')
import re
import time
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization
from tqdm.notebook import tqdm
import io
import torch
from cleantext import clean
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sahira\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\Sahira\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Sahira\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [8]:
#Gensim
import openpyxl
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
#import pyLDAvis.gensim_models

In [10]:
df = openpyxl.load_workbook("C:/Users/Sahira/OneDrive - Universiti Malaya/WIX3002/rotten_tomatoes_critic_reviews.xlsx")

In [12]:
type(df)

openpyxl.workbook.workbook.Workbook

In [11]:
df.info()

AttributeError: 'Workbook' object has no attribute 'info'

In [ ]:
data = dft.ctext.values.tolist()

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
tt = nltk.tokenize.TweetTokenizer()

def lemmatize_text(text):
    tokens = [lemmatizer.lemmatize(w, pos="v") for w in tt.tokenize(text)]
    return tokens

test = pd.DataFrame(['inviting rocks saw studying scheduled zoom meeting project', 'she likes these books', 'wow this is great'], columns=['text'])
test['text_lemmatized'] = test.text.apply(lemmatize_text)
test.head
#tt = nltk.tokenize.TweetTokenizer()
#tt.tokenize(text)
data_ready=dft['ctext'].apply(lemmatize_text)
dft['tokens'] = data_ready

In [ ]:
def create_dictionary(words):
    return corpora.Dictionary(words)
#passing the dataframe column having tokens as the argument
id2word=create_dictionary(dft.tokens)
print(id2word)

In [ ]:
def create_document_matrix(tokens,id2word):
    corpus = []
    for text in tokens:
       corpus.append(id2word.doc2bow(text))
    return corpus
#passing the dataframe column having tokens and dictionary
corpus=create_document_matrix(dft.tokens,id2word)
print(dft.tokens[0])
print(corpus[0])

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
 id2word=id2word,
 num_topics=10,
 random_state=100,
 )

In [ ]:
da_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

print(lda_model.print_topics())

In [ ]:
def get_lda_topics(model, num_topics, top_n_words):
     word_dict = {}
     for i in range(num_topics):
         word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in model.show_topic(i, topn = top_n_words)];
     return pd.DataFrame(word_dict)
get_lda_topics(lda_model,10,10)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis